# MLE応用：2標本比率の差の検定

このノートブックでは、**「2つの母比率の差の検定（Z検定）」** の構造を、最尤推定（MLE）の観点から掘り下げて解説します。

特に、以下の点に焦点を当てます。

1.  なぜ帰無仮説の下では **「プール推定（合算推定）」** を使うのが自然なのか。
2.  $\operatorname*{argmax}$ を用いた数式による、制約なしMLEと制約付きMLEの導出。
3.  **Wald検定・Score検定・尤度比検定（LRT）** の3つのアプローチの比較。
4.  2x2分割表の **独立性のカイ二乗検定** との関係。

## 1. 問題設定

2つのグループ（例：男性 $m$、女性 $f$）があり、それぞれのグループで「ある属性を持っているか（成功 $1$ / 失敗 $0$）」を観測します。

### データモデル
各個体のデータを $(G, Y)$ とします。
*   $G \in \{m, f\}$ : グループ（Group）
*   $Y \in \{0, 1\}$ : 結果（Outcome）

各グループの成功確率を $p_m, p_f$ と置きます。
$$ p_m = P(Y=1 \mid G=m), \quad p_f = P(Y=1 \mid G=f) $$

観測データは以下の集計値で表されます。
*   男性：試行数 $n_m$, 成功数 $x_m$
*   女性：試行数 $n_f$, 成功数 $x_f$

これらは互いに独立な二項分布に従うと仮定します。
$$ X_m \sim \mathrm{Bin}(n_m, p_m), \quad X_f \sim \mathrm{Bin}(n_f, p_f) $$

### 検定したい仮説
「男女で比率に差があるか」を検定します。

$$ H_0: p_m = p_f \quad (= p) \qquad \text{(帰無仮説：差がない)} $$
$$ H_1: p_m \neq p_f \qquad \text{(対立仮説：差がある)} $$

## 2. 尤度と最尤推定（argmaxによる導出）

検定統計量を構成するために、まずは **「尤度関数」** と、それを最大化する **「最尤推定量（MLE）」** を導出します。

### 尤度関数
独立性より、結合尤度は積で書けます。

$$ L(p_m, p_f \mid x_m, x_f) = \binom{n_m}{x_m} p_m^{x_m} (1-p_m)^{n_m - x_m} \times \binom{n_f}{x_f} p_f^{x_f} (1-p_f)^{n_f - x_f} $$

（定数項 $\binom{n}{x}$ は最大化に関係しないため、以下では省略することがあります）

### argmax の便利な性質

最尤推定を行う上で、以下の $\operatorname*{argmax}$ の性質（計算則）を知っておくと便利です。

#### (A) 定数倍は argmax を変えない
$c > 0$ のとき、
$$
\operatorname*{argmax}_{x} c f(x) = \operatorname*{argmax}_{x} f(x)
$$
（尤度関数の定数項 $\binom{n}{x}$ を無視できるのはこのためです）

#### (B) 単調増加変換は argmax を変えない
$g(u)$ が単調増加関数のとき、
$$
\operatorname*{argmax}_{x} g(f(x)) = \operatorname*{argmax}_{x} f(x)
$$
（対数尤度 $\log L$ を最大化してもよいのはこのためです）

#### (C) 定数加算は argmax を変えない
任意の定数 $c$ について
$$
\operatorname*{argmax}_{p \in D} f(p) = \operatorname*{argmax}_{p \in D} (f(p) + c)
$$
が成り立ちます。（理由：定数を足しても大小関係は変わらないため）

---

### A. 非制約モデル（$H_1$ の世界）

$p_m$ と $p_f$ が自由に動ける場合です。尤度関数は $p_m$ だけの項と $p_f$ だけの項の積に分離できるため、個別に最大化できます。

$$
\begin{aligned}
(\hat{p}_m, \hat{p}_f) &= \operatorname*{argmax}_{(p_m, p_f) \in [0,1]^2} L(p_m, p_f) \\
&= \left( \operatorname*{argmax}_{p_m} L_m(p_m), \quad \operatorname*{argmax}_{p_f} L_f(p_f) \right)
\end{aligned}
$$

それぞれの argmax は標本比率そのものになります。

$$ \hat{p}_m = \frac{x_m}{n_m}, \quad \hat{p}_f = \frac{x_f}{n_f} $$

これが **Wald検定** などで使われる、対立仮説側での推定値です。

---

### B. 制約付きモデル（$H_0$ の世界：プール推定）

帰無仮説 $H_0: p_m = p_f = p$ の下では、パラメータは共通の $p$ 一つになります。
この制約下での尤度 $L_0(p)$ は、データを「合算（プール）」したものになります。

$$
\begin{aligned}
L_0(p) &\propto p^{x_m} (1-p)^{n_m - x_m} \cdot p^{x_f} (1-p)^{n_f - x_f} \\
&= p^{x_m + x_f} (1-p)^{(n_m + n_f) - (x_m + x_f)}
\end{aligned}
$$

ここで、合計成功数 $S = x_m + x_f$、合計試行数 $N = n_m + n_f$ と置くと、これは試行数 $N$、成功数 $S$ の二項分布の尤度と同じ形になります。

したがって、制約付き最尤推定量 $\hat{p}_{pool}$ は以下のように求まります。

$$
\hat{p}_{pool} = \operatorname*{argmax}_{p \in [0,1]} L_0(p) = \frac{x_m + x_f}{n_m + n_f}
$$

これが **「プール推定（合算推定）」** です。
「$H_0$（男女差がない）を採用するなら、男女のラベルは確率推定に無関係な情報となるため、データを混ぜて推定するのが最も自然（最尤）」ということです。

## 2.1 argmaxの計算（高校数学の最大値問題として）

argmax を**高校数学の最大値問題**と完全に同じ形で書き下すと、見通しがかなり良くなります。
ポイントは次の3つです。

* **argmax は"最大値を与える点（入力）"を返す演算子**
* **log は単調増加なので、最大化する点（argmax）を変えない**
* あとは高校数学通りに **"微分=0 と端点"で決まる**

---

### 1) argmax / max を定義で書く

関数 $f: D 	o \mathbb{R}$ に対して

$$
\max_{p \in D} f(p)
$$

は**最大値（値そのもの）**、

$$
\operatorname*{argmax}_{p \in D} f(p)
$$

は**最大値を達成する点（入力）**です。

厳密には argmax は集合（複数解もありうる）なので

$$
\operatorname*{argmax}_{p \in D} f(p) := \{\,p \in D : f(p) = \max_{u \in D} f(u)\,\}
$$

と定義します（これが「気持ち悪さ」を消す定義）。
この定義からすぐに

* もし argmax が $\{p^*\}$ の1点集合なら、「解は $p^*$」と言ってよい
* $\max f = f(p^*)$ で、最大値はその点での関数値

が徹底されます。

---

### 2) この問題を max/argmax の形のまま進める

二項分布の尤度を

$$
L(p) := L(p \mid x) = \binom{n}{x} p^x (1-p)^{n-x}, \qquad p \in [0,1]
$$

と置きます（以後 $\mid x$ は省略）。
求めたいのは

$$
\hat{p} \in \operatorname*{argmax}_{p \in [0,1]} L(p)
$$

です（集合として書くのが最も正確）。

---

### 3) 「定数を捨てる」「log を取る」を argmax の等式で書く

#### (A) 正の定数倍は argmax を変えない

$\binom{n}{x} > 0$ なので

$$
\operatorname*{argmax}_{p \in [0,1]} L(p)
= \operatorname*{argmax}_{p \in [0,1]} \bigl[p^x (1-p)^{n-x}\bigr]
$$

（理由：正の定数倍は大小関係を変えないから）

一般形で書くと、任意の $c>0$ について

$$
\operatorname*{argmax}_{p \in D} f(p)
= \operatorname*{argmax}_{p \in D} \bigl(c\,f(p)\bigr)
$$

が成り立ちます。

#### (B) 単調増加変換は argmax を変えない（log の正当化）

$(0,1)$ では $p^x(1-p)^{n-x} > 0$ なので $\log$ が取れます。さらに $\log$ は単調増加なので

$$
\operatorname*{argmax}_{p \in (0,1)} \bigl[p^x (1-p)^{n-x}\bigr]
= \operatorname*{argmax}_{p \in (0,1)} \log\bigl(p^x (1-p)^{n-x}\bigr)
$$

一般形で、$\phi$ が単調増加なら

$$
\operatorname*{argmax}_{p \in D} f(p)
= \operatorname*{argmax}_{p \in D} \phi(f(p))
$$

が成り立ちます。

---

### 4) 端点 $p=0,1$ を含む形のまま扱う

ここで「端点が邪魔で log が取れない」問題が出ますが、最大化としてはこう整理できます。

* $x \in \{1,2,\dots,n-1\}$ のとき：端点では尤度 $L(0)=0,\ L(1)=0$ で、内部で正なので最大は内部にある。
  → 端点を気にせず $(0,1)$ で最大化してよい
* $x=0$ のとき：$L(p) \propto (1-p)^n$ なので最大は $p=0$
* $x=n$ のとき：$L(p) \propto p^n$ なので最大は $p=1$

以後まず主ケース $1 \le x \le n-1$ をやります（このとき解は内部）。

---

### 5) log 尤度の max/argmax を明示したまま微分に落とす

log 尤度を

$$
\ell(p) := \log L(p)
= \log \binom{n}{x} + x \log p + (n-x) \log(1-p)
$$

と定義。
すると、上の議論により（主ケースでは）

$$
\operatorname*{argmax}_{p \in [0,1]} L(p)
= \operatorname*{argmax}_{p \in (0,1)} \ell(p)
$$

ここまで「argmax の形」のまま来ています。

---

### 6) 高校数学の最大値判定を argmax の定義に接続する

$\ell(p)$ を微分すると

$$
\ell'(p) = \frac{x}{p} - \frac{n-x}{1-p}
$$

極値候補は $\ell'(p)=0$：

$$
\frac{x}{p} = \frac{n-x}{1-p}
\Longleftrightarrow x(1-p) = p(n-x)
\Longleftrightarrow x = np
\Longleftrightarrow p = \frac{x}{n}
$$

この点を $p^* := x/n$ と置く。

次に2階微分：

$$
\ell''(p) = -\frac{x}{p^2} - \frac{n-x}{(1-p)^2} < 0 \quad (0<p<1)
$$

これは $\ell$ が $(0,1)$ で**凹（concave）**であることを意味します。凹関数は「局所最大＝大域最大」なので、

* $\ell'(p^*)=0$ の解 $p^*$ は**唯一の最大点**
* したがって argmax は1点集合

になります。

よって定義に戻して

$$
\operatorname*{argmax}_{p \in (0,1)} \ell(p) = \left\{\frac{x}{n}\right\}
$$

従って

$$
\operatorname*{argmax}_{p \in [0,1]} L(p) = \left\{\frac{x}{n}\right\}
\qquad (1 \le x \le n-1)
$$

慣習的に「推定量はその要素」として

$$
\hat{p} = \frac{x}{n}
$$

と書きます。

---

### 7) 端点ケースも argmax で書き切る

まとめると

$$
\operatorname*{argmax}_{p \in [0,1]} L(p)
=
\begin{cases}
\{0\}, & x=0 \\
\left\{\dfrac{x}{n}\right\}, & 1 \le x \le n-1 \\
\{1\}, & x=n
\end{cases}
$$

なので「解」は結局いつも

$$
\hat{p} = \frac{x}{n}
$$

（端点も含めて一致）になります。

---

### 8) max と argmax の関係も式で固定

最後に「最大値（値）」も書くと、

$$
\max_{p \in [0,1]} L(p) = L(\hat{p})
$$

argmax が1点集合 $\{\hat{p}\}$ なら

$$
\operatorname*{argmax}_{p \in [0,1]} L(p) = \{\hat{p}\}
$$

という対応が完全に揃います。

---

必要なら次に、同じ書式のまま（argmax を保ったまま）制約付き（プール）の

$$
\hat{p}_{\text{pool}}
\in \operatorname*{argmax}_{p \in [0,1]}
\Bigl[p^{x_m+x_f}(1-p)^{(n_m-x_m)+(n_f-x_f)}\Bigr]
$$

を同じ微分で解いて $(x_m+x_f)/(n_m+n_f)$ に落とすところまで、完全に同じ型で並べられます。


## 3. 3つの検定手法の比較（Wald / Score / LRT）

2標本比率の差の検定には、近似の作り方によって主に3つの流派があります。これらは大標本では一致しますが、**「分散（標準誤差）をどこで評価するか」** が異なります。

### 1) Wald検定（非プール）
「推定した差」を「推定した標準誤差（$H_1$側）」で割る方法です。

$$ Z_W = \frac{\hat{p}_m - \hat{p}_f}{\sqrt{\frac{\hat{p}_m(1-\hat{p}_m)}{n_m} + \frac{\hat{p}_f(1-\hat{p}_f)}{n_f}}} $$

*   特徴：分母に **個別の推定量 $\hat{p}_m, \hat{p}_f$** を使います。
*   直感：推定精度の信頼区間をベースにした検定。

### 2) Score検定（プール）
「帰無仮説 $H_0$ が正しい世界」での標準誤差で標準化する方法です。統計検定などで正解とされる「①の式」はこれです。

$$ Z_S = \frac{\hat{p}_m - \hat{p}_f}{\sqrt{\hat{p}_{pool}(1-\hat{p}_{pool}) \left(\frac{1}{n_m} + \frac{1}{n_f}\right)}} $$

*   特徴：分母に **プール推定量 $\hat{p}_{pool}$** を使います。
*   直感：$H_0$ の世界で想定される「ノイズの大きさ」で、観測された「ズレ」を評価する。
*   **重要**：Pearsonのカイ二乗検定と（2x2の場合）等価です（$Z_S^2 = \chi^2$）。

### 3) 尤度比検定（LRT）
制約なし最大尤度と、制約あり最大尤度の比（高さの比）を見る方法です。

$$ \Lambda = \frac{\max_{p} L_0(p)}{\max_{p_m, p_f} L(p_m, p_f)} = \frac{L_0(\hat{p}_{pool})}{L(\hat{p}_m, \hat{p}_f)} $$

検定統計量（逸脱度）:
$$ G^2 = -2 \log \Lambda \sim \chi^2_1 $$

*   特徴：推定量の差ではなく、尤度の比を直接使います。

---

### 3手法の比較表

| 観点 | Wald（非プール） | Score（プール） | LRT（尤度比） |
| :--- | :--- | :--- | :--- |
| **位置づけ** | 推定ベースの検定 | **$H_0$ 整合** の検定 | モデル比較（尤度の高さ） |
| **使うモデル** | 非制約モデル（$p_m, p_f$） | 制約モデル（$p$） | 両方 |
| **MLEの利用** | 分母に **個別MLE** | 分母に **プールMLE** | 分子・分母にそれぞれのMLE |
| **2x2表との関係** | 近いが一致しない | **Pearson $\chi^2$ と一致** ($Z^2 = \chi^2$) | G検定（$G^2$）と一致 |
| **特徴** | 計算が楽、信頼区間と整合 | 検定として筋が良い | 最尤原理に忠実、一般化容易 |

## 4. 実装例：具体的な数値での確認

以下のデータを用いて、実際に3つの検定統計量を計算し、Score検定（プールZ）の二乗がカイ二乗値と一致することを確認します。

*   男性：$n_m = 111, x_m = 38$
*   女性：$n_f = 106, x_f = 60$

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import japanize_matplotlib

# 1. データ設定
n_m, x_m = 111, 38
n_f, x_f = 106, 60

# 2. 推定量の計算
p_m_hat = x_m / n_m
p_f_hat = x_f / n_f
p_pool_hat = (x_m + x_f) / (n_m + n_f)

print(f"標本比率(男): {p_m_hat:.4f}")
print(f"標本比率(女): {p_f_hat:.4f}")
print(f"プール比率  : {p_pool_hat:.4f}")
print("-" * 30)

# 3. 検定統計量の計算

# (A) Wald検定（非プール分散）
var_wald = (p_m_hat * (1 - p_m_hat) / n_m) + (p_f_hat * (1 - p_f_hat) / n_f)
z_wald = (p_m_hat - p_f_hat) / np.sqrt(var_wald)

# (B) Score検定（プール分散）
var_score = p_pool_hat * (1 - p_pool_hat) * (1/n_m + 1/n_f)
z_score = (p_m_hat - p_f_hat) / np.sqrt(var_score)

# (C) 尤度比検定 (LRT)
def log_likelihood(n, x, p):
    # p=0,1 の回避
    p = np.clip(p, 1e-10, 1 - 1e-10)
    return x * np.log(p) + (n - x) * np.log(1 - p)

ll_h1 = log_likelihood(n_m, x_m, p_m_hat) + log_likelihood(n_f, x_f, p_f_hat)
ll_h0 = log_likelihood(n_m, x_m, p_pool_hat) + log_likelihood(n_f, x_f, p_pool_hat)
g_squared = -2 * (ll_h0 - ll_h1)

# (D) Pearsonのカイ二乗検定 (scipy.stats)
obs = np.array([[x_m, n_m - x_m], [x_f, n_f - x_f]])
chi2, p_val, dof, expected = stats.chi2_contingency(obs, correction=False)

# 結果表示
print(f"Wald Z      : {z_wald:.4f}  => Z^2: {z_wald**2:.4f}")
print(f"Score Z     : {z_score:.4f}  => Z^2: {z_score**2:.4f}")
print(f"Pearson Chi2: {chi2:.4f}")
print(f"LRT G^2     : {g_squared:.4f}")

print("-" * 30)
print(f"一致確認: Score Z^2 ({z_score**2:.4f}) vs Pearson Chi2 ({chi2:.4f})")
if np.isclose(z_score**2, chi2):
    print(" >> 完全一致しました（理論通り）")
else:
    print(" >> 不一致（計算誤差の可能性あり）")


## 5. 可視化：比率の比較と検定統計量

結果を視覚的に理解するために、以下のグラフを作成します。
1.  **標本比率の棒グラフ**：男性・女性・プール推定の比率を比較。
2.  **標準正規分布とZ値**：Score検定のZ値が帰無分布（標準正規）のどこに位置するか。
3.  **カイ二乗分布と検定統計量**：$Z^2$ や $G^2$ がカイ二乗分布のどこに位置するか。

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. 比率の棒グラフ
labels = ['男性 (Male)', '女性 (Female)', 'プール (Pooled)']
values = [p_m_hat, p_f_hat, p_pool_hat]
colors = ['skyblue', 'pink', 'lightgreen']

axes[0].bar(labels, values, color=colors, alpha=0.8)
axes[0].set_ylim(0, 1)
axes[0].set_ylabel('比率 (Proportion)')
axes[0].set_title('標本比率とプール推定')
for i, v in enumerate(values):
    axes[0].text(i, v + 0.02, f"{v:.3f}", ha='center', fontweight='bold')

# 2. 標準正規分布とZ値 (Score Test)
x = np.linspace(-4, 4, 1000)
y = stats.norm.pdf(x, 0, 1)
axes[1].plot(x, y, label='標準正規分布 $N(0,1)$', color='gray')
axes[1].fill_between(x, y, where=(x <= -1.96) | (x >= 1.96), color='red', alpha=0.2, label='棄却域 (5%)')
axes[1].axvline(z_score, color='blue', linestyle='--', linewidth=2, label=f'Score Z = {z_score:.2f}')
axes[1].set_title('Score検定統計量 $Z_S$ の位置')
axes[1].legend()

# 3. カイ二乗分布と検定統計量
x_chi = np.linspace(0, 15, 1000)
y_chi = stats.chi2.pdf(x_chi, df=1)
axes[2].plot(x_chi, y_chi, label='$\\chi^2_1$ 分布', color='gray')
axes[2].fill_between(x_chi, y_chi, where=(x_chi >= 3.84), color='red', alpha=0.2, label='棄却域 (5%)')
axes[2].axvline(z_score**2, color='blue', linestyle='--', linewidth=2, label=f'$Z_S^2 = \\chi^2 = {z_score**2:.2f}$')
axes[2].set_title('カイ二乗統計量の位置')
axes[2].set_ylim(0, 0.5)
axes[2].legend()

plt.tight_layout()
plt.show()

## 6. 「独立」の2つの意味

この問題で混乱しやすい「独立」という言葉には、2つの異なる文脈があります。

1.  **標本の独立性（Sampling Independence）**
    *   「男性グループのデータ」と「女性グループのデータ」が互いに影響しないこと。
    *   これにより、分散の加法性が成り立ちます：
        $\mathrm{Var}(\hat{p}_m - \hat{p}_f) = \mathrm{Var}(\hat{p}_m) + \mathrm{Var}(\hat{p}_f)$

2.  **変数間の独立性（Independence of Variables）**
    *   帰無仮説 $H_0$ そのものの意味です。「性別ラベル $G$ と 結果 $Y$ が独立である（関連がない）」こと。
    *   $Y \perp G \iff P(Y=1|G=m) = P(Y=1|G=f)$。
    *   この仮説を採用するからこそ、「ラベルを外して合算（プール）」する操作が正当化されます。

Score検定では、2の意味での独立性を仮定した上で（プール推定を行い）、1の意味での独立性を使って分散を計算している、という構造になっています。

## 7. インタラクティブ・シミュレーション

スライダーを動かして、サンプルサイズや成功数を変えたときに検定結果がどう変化するか確認してみましょう。

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output


# インタラクティブな可視化
def interactive_proportion_test(n_m, x_m, n_f, x_f):
    # 推定
    p_m_hat = x_m / n_m if n_m > 0 else 0
    p_f_hat = x_f / n_f if n_f > 0 else 0
    p_pool = (x_m + x_f) / (n_m + n_f) if (n_m + n_f) > 0 else 0
    
    # Score Z
    denom = np.sqrt(p_pool * (1 - p_pool) * (1/n_m + 1/n_f)) if (n_m > 0 and n_f > 0 and p_pool > 0 and p_pool < 1) else 1e-10
    z_score = (p_m_hat - p_f_hat) / denom
    p_val = 2 * (1 - stats.norm.cdf(abs(z_score)))
    
    # グラフ描画
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    
    # 比率
    ax[0].bar(['Male', 'Female', 'Pooled'], [p_m_hat, p_f_hat, p_pool], color=['blue', 'red', 'green'], alpha=0.6)
    ax[0].set_ylim(0, 1)
    ax[0].set_title(f'Proportions (pm={p_m_hat:.3f}, pf={p_f_hat:.3f})')
    
    # Z分布
    x = np.linspace(-4, 4, 100)
    y = stats.norm.pdf(x)
    ax[1].plot(x, y, 'k-', lw=2)
    ax[1].fill_between(x, y, where=(x <= -1.96) | (x >= 1.96), color='red', alpha=0.2)
    ax[1].axvline(z_score, color='blue', linestyle='--', label=f'Z={z_score:.3f}')
    ax[1].set_title(f'Score Test (p-val={p_val:.4f})')
    ax[1].legend()
    
    plt.tight_layout()
    display(fig)
    plt.close(fig)

# Widgets
w_nm = widgets.IntSlider(value=100, min=10, max=500, description='n_m')
w_xm = widgets.IntSlider(value=40, min=0, max=500, description='x_m')
w_nf = widgets.IntSlider(value=100, min=10, max=500, description='n_f')
w_xf = widgets.IntSlider(value=50, min=0, max=500, description='x_f')

def update_x_max(*args):
    w_xm.max = w_nm.value
    w_xf.max = w_nf.value

w_nm.observe(update_x_max, 'value')
w_nf.observe(update_x_max, 'value')

ui = widgets.VBox([
    widgets.HBox([w_nm, w_xm]),
    widgets.HBox([w_nf, w_xf])
])

out = widgets.interactive_output(interactive_proportion_test, {'n_m': w_nm, 'x_m': w_xm, 'n_f': w_nf, 'x_f': w_xf})
display(ui, out)
